In [3]:
import pandas as pd

In [9]:
df_main = pd.read_pickle("df_main_fe.pkl")
df_test_main = pd.read_pickle("df_test_main_fe.pkl")

In [10]:
df = df_main

In [6]:
threshold = 50
high_cardinality_cols = [col for col in df.select_dtypes(include=['object']).columns if df[col].nunique() > threshold]

In [7]:
for col in high_cardinality_cols:
    df[col], _ = df[col].factorize()

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

inputs = []
embeddings = []

for col in high_cardinality_cols:
    # Input layer for each categorical column
    input_col = Input(shape=(1,))
    inputs.append(input_col)
    
    # Calculate embedding size. Adjust the values as needed.
    unique_vals = df[col].nunique()
    embedding_size = min(50, unique_vals // 2)
    
    # Embedding layer for each categorical column
    embedded_col = Embedding(input_dim=unique_vals, output_dim=embedding_size)(input_col)
    embedded_col = Flatten()(embedded_col)  # Flatten the embedding layer
    embeddings.append(embedded_col)

# Concatenate all embedding layers
x = Concatenate()(embeddings)

# Add additional dense layers if needed
x = Dense(128, activation='relu')(x)
# ... (you can add more layers as needed)

# Assuming a binary classification task for simplicity
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


ModuleNotFoundError: No module named 'tensorflow'